In [ ]:
import tensorflow as tf

In [1]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow import keras
#from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
#from keras.optimizers import Adam
#from Data_Preparation import *
#from keras.utils import to_categorical
from scipy.spatial import distance_matrix
import os
import cv2
import numpy as np
import SimpleITK as sitk
import glob
import os
from scipy.ndimage import distance_transform_cdt
import pickle
from matplotlib import pyplot as plt
#import matplotlib.image as mpimg
import imutils
%matplotlib inline
import pandas as pd

## Loading the  Model

In [2]:
from tensorflow import keras
from keras.models import load_model
model = keras.models.load_model('//ibs9010/current_data/Data_Keshav/image/Classification_model/NeuronClassifierModels/BinaryClassifier_trial5_bigmodel.h5')

Using TensorFlow backend.


In [ ]:
# Function to determine local coordinates in a tile that has been classified as to having containing Neurons
def local_coordinates(file):
    x_coordinates = [] #x-coordinate
    y_coordinates = [] #y-coordinate
    z_coordinates = [] #y-coordinate
    coordinates = [] #x,y-coordinate put together
    #print(file)
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.imshow(img,cmap = "gray")
    plt.show()

    # global thresholding to create binarized image
    ret1,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    # Otsu's thresholding
    ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #plt.imshow(th2,cmap = "gray")
    #plt.show()

    #Gaussian blurring to smoothen the image
    #blur_image = cv2.GaussianBlur(th2,(5,5),0)
    blur_image = cv2.medianBlur(th2,5)
    #blur_image = cv2.bilateralFilter(th2,9,75,75)
    #print("Blur image with bilat filter")
    #plt.imshow(blur_image,cmap = "gray")
    #plt.show()

    # calculating the centre of the objects found in the image
    
    # Convert image into a np array
    npimage = np.array(blur_image)
    
    if npimage.any():
        indsx,indsy = np.where(npimage>0)
        xmean,ymean = [indsy.mean(),indsx.mean()]
        #print(x,y)
        Cx,Cy = int(xmean), int(ymean)
        cv2.circle(img, (Cx, Cy), 1, (255, 0, 0), 1)
        #plt.imshow(img,cmap = "gray")
        #plt.show()
    else:
        xmean=0
        ymean =0
    # show the image
    #plt.imshow(img,cmap = "gray")
    #plt.show()
    
    return xmean,ymean

## Blob Detection with contour and centroid using opencv methods

In [44]:
# Function to determine local coordinates in a tile that has been classified as to having containing Neurons
def blob_local_coordinates(file):
    x_coordinates = [] #x-coordinate
    y_coordinates = [] #y-coordinate
    z_coordinates = [] #y-coordinate
    coordinates = [] #x,y-coordinate put together
    print(file)
    img = cv2.imread(file)
    # Convert the image into gray scale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #img_gray = cv2.imread(file)
    #plt.imshow(img_gray,cmap = "gray")
    #plt.show()

    # global thresholding to create binarized image
    ret1,th1 = cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)
    # Adding filter of Otsu's thresholding to binarized image
    ret2,th2 = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #plt.imshow(th2,cmap = "gray")
    #plt.show()

    #Gaussian blurring to smoothen the thresholded image to remove salt and pepper noise and improve accuracy
    bilat_image = cv2.bilateralFilter(th2,9,75,75)
    blur_image = cv2.medianBlur(bilat_image,5)
    print("Blur image with bilat and median filter")
    #plt.imshow(blur_image,cmap = "gray")
    #plt.show()
    
    npimage = np.array(blur_image)
    xlocal = 0
    ylocal = 0
    
    # calculating the centre of the objects found in the image

    if npimage.any():
        indsx,indsy = np.where(npimage>0)
        xmean,ymean = [indsy.mean(),indsx.mean()]
        xmean,ymean = int(xmean), int(ymean)
        print("xmean,ymean:",xmean,ymean)
        
        if xmean >=0 and ymean>=0:
            # find contours in the thresholded image
            cnts = cv2.findContours(blur_image, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)

            # loop over the contours
            for c in cnts:
                # compute the center of the contour
                M = cv2.moments(c)
                #print(M)
                area = cv2.contourArea(c)
                print("area:",area)
                if area >130:
                    cX = int(M["m10"] / (M["m00"]+ 1e-5))
                    cY = int(M["m01"] / (M["m00"]+ 1e-5))
                    print("cX,cY:",cX,cY)
                    xlocal = cX
                    ylocal = cY
                    # draw the contour and center of the shape on the image
                    cv2.drawContours(img, [c], -1, (0, 255, 0), 1)
                    cv2.circle(img, (cX, cY), 1, (255, 0, 0), 1)
                    # show the image
                    #plt.imshow(img)
                    #plt.show()
    else:
        xlocal = 0
        ylocal = 0
        
    return xlocal,ylocal

In [4]:
# Function for Creating LandmarkcAscii Files
def writeLandmarkCoords(filename, landmarkList):
  with open(filename, 'w') as file:
     file.write("# Avizo 3D ASCII 2.0 \n")
     file.write("define Markers ")
     file.write("{}".format(len(landmarkList)))
     file.write("\n")
     file.write("Parameters {\n")
     file.write("    NumSets 1,\n")
     file.write("    ContentType \"LandmarkSet\"\n")
     file.write("}\n")

     file.write("Markers { float[3] Coordinates } @1\n")

     file.write("# Data section follows\n")
     file.write("@1\n")
     for landmark in landmarkList:
         file.write("{}".format(landmark[0]))
         file.write(" ")
         file.write("{}".format(landmark[1]))
         file.write(" ")
         file.write("{}".format(landmark[2]))
         file.write("\n")

In [5]:
def read_landmarks(filename,only_2d=True):
    manual_landmark_list = []
    lines = []
    with open(filename, 'r') as csb:
        lines = csb.readlines()
    number = 0

    # for each manual landmark
    for line in lines:
        #print line
        if line.startswith("@1"):
            number = 1
            continue
        if number == 1 and line.isspace() == False:
            #print line.split()
            pt = list(map(float,line.split()))
            #pt = [pt[0]*self.axis_directions[0],pt[1]*self.axis_directions[1],pt[2]*self.axis_directions[2]]
            manual_landmark_list.append([pt[0],pt[1]])

    return manual_landmark_list

In [6]:
def write_landmarks(landmarks,filename):

    with open(filename, 'w') as file:
        file.write("# Avizo 3D ASCII 2.0 \n")
        file.write("define Markers ")
        file.write("{}".format(len(landmarks)))
        file.write("\n")
        file.write("Parameters {\n")
        file.write("    NumSets 1,\n")
        file.write("    ContentType \"LandmarkSet\"\n")
        file.write("}\n")

        file.write("Markers { float[3] Coordinates } @1\n")

        file.write("# Data section follows\n")
        file.write("@1\n")

        if len(landmarks)!=0:
            for landmark in landmarks:
                ##print(landmark[0][0])
                file.write("{}".format(landmark[0]))
                file.write(" ")
                file.write("{}".format(landmark[1]))
                file.write(" ")
                if len(landmark) > 2:
                    file.write("{}".format(landmark[2]))
                else:
                    file.write("{}".format(0))
                file.write("\n")


## Running Inference

In [45]:
output = list()
percentage_output = list()
master_df = pd.DataFrame(columns =['slice_number','manual_landmark_count','TP','FN','FP','TP_per','FN_per','FP_per'])
for x in range(51,52):
    print(x)
    if x in[35,80,81]:# MG48
        continue
    #if x in[68,72,90]:# MG49
    #    continue    
    img_size = 70 # fix the size of the image
    #Path = "//ibs9010/current_data/Data_Keshav/Cropped_Images/MG48_3day_bs/S{}/".format(x)
    Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images/newinference/newS51/'
    Testing_image = []# list of testing images
    image_name = []# list of image names

    for img in os.listdir(Path): #gives the images from the path
        img_array = cv2.imread(os.path.join(Path,img),cv2.IMREAD_GRAYSCALE) # read every image in gray scale from the given path
        Testing_image.append(img_array)
        image_name.append(img)

    print("Number of test images:",len(Testing_image))
    #Running Inference
    try:
        Test_image = np.array(Testing_image).reshape(-1,img_size,img_size,1)
    except ValueError:
        print(" cannot reshape array of size 4860 into shape (70,70,1)")
        continue    
    
    #make an array of every element of list from Testing_image and then reshape them
    Test_image = Test_image/255.0
    scores = model.predict_classes(Test_image)

    # Results
    neuron_image_counter = bkgd_image_counter = 0
    for i in range(len(Test_image)):
        #print("Image:",image_name[i],"with score:",scores[i]) 
        if scores[i]==0:
            bkgd_image_counter +=1
        if scores[i]==1:
            neuron_image_counter += 1

    print("Total testing images:",len(Testing_image))
    print("bkgd_image_counter:",bkgd_image_counter,"neuron_image_counter:",neuron_image_counter)


    ## Saving classified tiles into respective folders
    dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S{}/'.format(x)
    directory = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Data/Test/'
    
    if not os.path.exists(dest_path):
       print("path doesn't exist. trying to make the folder.")
       os.makedirs(dest_path)
    
    # Delete any exiting images before saving
    import glob
    os.chdir(dest_path)
    files=glob.glob('*.tif')
    for filename in files:
        os.remove(filename)

    single_image_list  = [] # contains list of filenames of single neurons
    for k in range(1,len(Test_image)):
        if scores[k]==1:
            single_image_list.append(image_name[k])

    for i in range(len(Test_image)):
        if scores[i]==1:
            cv2.imwrite(os.path.join(dest_path,image_name[i]), Test_image[i]*255)
            #pass

    # Generating pickle file For Single neurons
    import pickle
    with open('//ibs9010/current_data/Data_Keshav/image/Classification_model/27aug_MG48_manual_S{:03d}.pkl'.format(x), 'wb') as f:
        pickle.dump(single_image_list, f)        

    # Landmarks Generator
    # Path to obatin the classified tiles
    path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/*'
    output_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/landmarks/'

    # Pickle files in order to obtain (from the Cropping function to get) Vertices and tile counter data and also list of neuron tiles after classification
    with open('//ibs9010/current_data/Data_Keshav/image/Classification_model/27aug_MG48_manual_S{:03d}.pkl'.format(x),'rb') as f:
        loaded_obj = pickle.load(f)
    with open('//ibs9010/current_data/Data_Keshav/image/Classification_model/global_xy_MG48_3day_bs_S51_multiiteration_piamasked.pkl'.format(x),'rb') as g:
        loaded_data = pickle.load(g)#cropped_data
    #with open('//ibs9010/current_data/Data_Keshav/image/Classification_model/PickleFiles/global_xy_MG48_bs_S{}_bigtile_piamasked.pkl'.format(x),'rb') as g:
    #    loaded_data = pickle.load(g)#cropped_data global_xy_MG48_3day_bs_S51_bigtile_piamasked
    
    dest_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S{}/'.format(x)
    global_list = []
    coordinates = []
    #print(loaded_obj)
    #print(loaded_data)
    
    for item in loaded_obj:
        local_x, local_y = blob_local_coordinates(dest_path+item)
        #local_x, local_y = local_coordinates(dest_path+item)
        
        filename= item[:-4]
        #print(filename)
        num = loaded_data[loaded_data['Tile_Counter'] == int(filename)].index[0]
        #num = loaded_data[loaded_data['Correct_Tile_Counter'] == int(filename) ].index[0]
        global_list = loaded_data.at[num, 'Vertices']
        #print(num,global_list)
        global_x = global_list[3]+  local_x
        global_y = global_list[0] + local_y
        globals = [global_x,global_y]
        coordinates.append(globals)

    # Merging landmarks for partials!
    from math import sqrt
    templist =[]
    newcoordinatelist = []

    for index_p1,p1 in enumerate(coordinates):
        for p2 in coordinates[index_p1+1:]:
            x2 = p2[0]
            x1 = p1[0]
            y2 = p2[1]
            y1 = p1[1]
            #print(x2,x1)
            #print(y2,y1)
            dist = sqrt((x2 - x1)** 2.0 + (y2 - y1)**2.0)

            if dist <20:
                #print(dist)
                templist.append(p1)
                templist.append(p2)
                avg = [(p1[0]+ p2[0])/2, (p1[1]+ p2[1])/2]
                #print(avg)
                newcoordinatelist.append(avg)
    for p in coordinates:
        if p not in templist:
            newcoordinatelist.append(p)
    
     #Removing duplicates in coordinates 
    print("Number of coordinates:",len(newcoordinatelist))
    newcoordinatelist = [c1 for idx, c1 in enumerate(newcoordinatelist) if c1 not in newcoordinatelist[:idx]]
    print("Filtered Number of coordinates:",len(newcoordinatelist))
    
    '''
    temp1list = []
    landmarklist = []
    
    for index_q1,q1 in enumerate(newcoordinatelist):
        for q2 in newcoordinatelist[index_p1+1:]:
            X2 = q2[0]
            X1 = q1[0]
            Y2 = q2[1]
            Y1 = q1[1]
            #print(x2,x1)
            #print(y2,y1)
            dist1 = sqrt((X2 - X1)** 2.0 + (Y2 - Y1)**2.0)

            if dist1 <20:
                #print(dist)
                temp1list.append(q1)
                temp1list.append(q2)
                avg1 = [(q1[0]+ q2[0])/2, (q1[1]+ q2[1])/2]
                #print(avg1)
                landmarklist.append(avg1)
    for q in newcoordinatelist:
        if q not in temp1list:
            landmarklist.append(q)
    
    print("Number of landmarklist coordinates:",len(landmarklist))
    #print(landmarklist)
'''
    #adding zero for z dimension for landmark files:
    for item in newcoordinatelist:
        item.append(0.5)
        
    # Fixing the resolutionm of the landmarks for Amira
    modified_coordinates= []
    for new in newcoordinatelist:
        x_modified = new[0]*0.868
        y_modified = new[1]*0.868
        z_modified = new[2]*0.868
        modified_coordinates.append([x_modified,y_modified,z_modified])

    #Calling function to create respective landmark file
    writeLandmarkCoords(output_path+"27aug_Manual_MG48_binary_S{:03d}.landmarkAscii".format(x), modified_coordinates)

    # TP, FP, FN Calculator 
    manual_landmarks_file = '//ibs9010/current_data/Data_Keshav/image/Classification_model/manual_landmark_files/MG48/S{:03d}_Landmarks.landmarkAscii'.format(x)
    auto_landmarks_file_full="//ibs9010/current_data/Data_Keshav/image/Classification_model/landmarks/27aug_Manual_MG48_binary_S{:03d}.landmarkAscii".format(x)

    error_resolution = 30 # radius around manual landmark where we compare , in microns
    output_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/LMfiles/'
    exp = 'S{:03d}'.format(x)

    manual_landmarks = read_landmarks(manual_landmarks_file)
    auto_landmarks_full = read_landmarks(auto_landmarks_file_full)
    
    auto_landmarks_all = []
    for pt in auto_landmarks_full:
        auto_landmarks_all.append(pt)

    manual_np = np.array(manual_landmarks)
    auto_np = np.array(auto_landmarks_all)
    confusion_mat = (distance_matrix(manual_np,auto_np) < error_resolution)

    TP = (confusion_mat.sum(axis=1)>0).sum()
    FN = (confusion_mat.sum(axis=1)==0).sum()
    FP = (confusion_mat.sum(axis=0)==0).sum()

    TP_per = (TP/len(manual_landmarks))*100
    FN_per = (FN/len(manual_landmarks))*100
    FP_per = (FP/len(manual_landmarks))*100

    TP_locations = np.where(confusion_mat.sum(axis=1))
    FN_locations = np.where(confusion_mat.sum(axis=1)==0)
    FP_locations = np.where(confusion_mat.sum(axis=0)==0)

    write_landmarks(manual_np[TP_locations],output_path+exp+'_Matches(TP).landmarksAscii')
    write_landmarks(manual_np[FN_locations],output_path+exp+'_Misses(FN).landmarksAscii')
    write_landmarks(auto_np[FP_locations],output_path+exp+'_FP.landmarksAscii')

    print("TP,FN,FP:",TP,FN,FP)
    print("TP_per,FN_per,FP_per:",TP_per,FN_per,FP_per)
    output.append([x,len(manual_landmarks),TP,FN,FP,TP_per,FN_per,FP_per])
    #percentage_output.append([x,len(manual_landmarks),TP_per,FN_per,FP_per])
    per_iteration = pd.DataFrame(data=[[x,len(manual_landmarks),TP,FN,FP,TP_per,FN_per,FP_per]],columns =['slice_number','manual_landmark_count','TP','FN','FP','TP_per','FN_per','FP_per'])
    master_df.append(per_iteration)
    #break
    
values = []
percentages = []
manual_lm = []
values.append([TP,FN,FN,len(manual_landmarks)])
percentages.append([TP_per,FN_per,FP_per])
#output = [[x,len(manual_landmarks),TP,FN,FP]]
stats = pd.DataFrame(output,columns =['slice_number','manual_landmark_count','TP','FN','FP','TP_per','FN_per','FP_per'])
#stats = pd.DataFrame(output,columns =['slice_number','manual_landmark_count','TP_per','FN_per','FP_per'])
#percent_stats = pd.DataFrame(output,columns =['slice_number','manual_landmark_count','TP_per','FN_per','FP_per'])
print(stats)
print(master_df)
file_name = '//ibs9010/current_data/Data_Keshav/MG48stats_S51.csv'
stats.to_csv(file_name, encoding='utf-8', index=False)
#file_name = '//ibs9010/current_data/Data_Keshav/MG48percentstats_Section1to20.csv'
#percent_stats.to_csv(file_name, encoding='utf-8', index=False)

51
Number of test images: 9408
Total testing images: 9408
bkgd_image_counter: 8951 neuron_image_counter: 457
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/1865.tif
Blur image with bilat and median filter
xmean,ymean: 53 31
area: 0.0
area: 9.5
area: 41.0
area: 145.5
cX,cY: 64 30
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/2025.tif
Blur image with bilat and median filter
xmean,ymean: 37 42
area: 39.0
area: 35.0
area: 4.0
area: 298.5
cX,cY: 5 55
area: 0.0
area: 0.0
area: 779.5
cX,cY: 49 42
area: 2.0
area: 4.5
area: 37.0
area: 18.5
area: 60.5
area: 11.5
area: 16.5
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/2371.tif
Blur image with bilat and median filter
xmean,ymean: 15 5
area: 4.5
area: 276.0
cX,cY: 14 4
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/Bi

xmean,ymean: 26 59
area: 304.0
cX,cY: 26 58
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/6316.tif
Blur image with bilat and median filter
xmean,ymean: 10 26
area: 0.0
area: 9.0
area: 246.0
cX,cY: 9 25
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7151.tif
Blur image with bilat and median filter
xmean,ymean: 53 58
area: 2.0
area: 423.0
cX,cY: 54 58
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/1616.tif
Blur image with bilat and median filter
xmean,ymean: 66 35
area: 58.5
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/2835.tif
Blur image with bilat and median filter
xmean,ymean: 35 43
area: 3.0
area: 16.0
area: 4.5
area: 152.5
cX,cY: 64 58
area: 90.5
area: 119.5
area: 57.0
//ibs9010/current_data/Data_Keshav/image

xmean,ymean: 26 48
area: 0.0
area: 0.0
area: 145.5
cX,cY: 20 59
area: 11.0
area: 0.5
area: 25.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/3912.tif
Blur image with bilat and median filter
xmean,ymean: 16 43
area: 0.0
area: 634.5
cX,cY: 16 42
area: 0.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/6945.tif
Blur image with bilat and median filter
xmean,ymean: 7 38
area: 44.0
area: 1.5
area: 643.0
cX,cY: 6 40
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7474.tif
Blur image with bilat and median filter
xmean,ymean: 17 38
area: 40.5
area: 83.0
area: 255.0
cX,cY: 9 35
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7481.tif
Blur image with bilat and median filter
xmean,ymean: 20 43
area: 476.0
cX,cY: 18 58
area: 0.

Blur image with bilat and median filter
xmean,ymean: 20 16
area: 140.5
cX,cY: 20 16
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/8488.tif
Blur image with bilat and median filter
xmean,ymean: 18 62
area: 184.0
cX,cY: 20 63
area: 16.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/8801.tif
Blur image with bilat and median filter
xmean,ymean: 29 40
area: 0.0
area: 2.0
area: 21.0
area: 815.0
cX,cY: 28 46
area: 12.5
area: 0.0
area: 4.0
area: 112.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/3489.tif
Blur image with bilat and median filter
xmean,ymean: 30 14
area: 0.0
area: 48.5
area: 489.5
cX,cY: 26 9
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/2450.tif
Blur image with bilat and median filter
xmean,ymean: 45 43


Blur image with bilat and median filter
xmean,ymean: 8 31
area: 58.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/6907.tif
Blur image with bilat and median filter
xmean,ymean: 42 38
area: 3.5
area: 31.5
area: 230.0
cX,cY: 58 63
area: 19.0
area: 9.0
area: 273.0
cX,cY: 34 24
area: 41.0
area: 26.0
area: 22.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7372.tif
Blur image with bilat and median filter
xmean,ymean: 42 50
area: 6.5
area: 33.5
area: 646.0
cX,cY: 50 59
area: 46.5
area: 6.5
area: 70.0
area: 18.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/8056.tif
Blur image with bilat and median filter
xmean,ymean: 38 32
area: 0.0
area: 13.0
area: 22.5
area: 2.0
area: 62.0
area: 6.5
area: 14.0
area: 5.5
area: 0.0
area: 22.5
area: 0.0
area: 348.5
cX,cY: 5 46
area: 7.5
area: 0.0
area: 

...

Blur image with bilat and median filter
xmean,ymean: 43 29
area: 5.0
area: 0.0
area: 355.5
cX,cY: 55 45
area: 1.0
area: 426.0
cX,cY: 34 12
area: 0.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/3910.tif
Blur image with bilat and median filter
xmean,ymean: 41 29
area: 0.5
area: 8.0
area: 0.0
area: 137.0
cX,cY: 53 42
area: 37.5
area: 0.5
area: 115.0
area: 6.5
area: 126.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7107.tif
Blur image with bilat and median filter
xmean,ymean: 44 47
area: 1.0
area: 137.5
cX,cY: 29 59
area: 116.5
area: 6.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/8481.tif
Blur image with bilat and median filter
xmean,ymean: 42 28
area: 13.5
area: 22.0
area: 23.0
area: 127.5
area: 30.5
area: 349.5
cX,cY: 48 24
area: 216.5
cX,cY: 51 6
//ibs9010/current_data/Data

Blur image with bilat and median filter
xmean,ymean: 28 51
area: 394.0
cX,cY: 28 51
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/2800.tif
Blur image with bilat and median filter
xmean,ymean: 33 15
area: 19.5
area: 25.0
area: 422.0
cX,cY: 34 12
area: 8.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/3396.tif
Blur image with bilat and median filter
xmean,ymean: 23 34
area: 140.5
cX,cY: 43 64
area: 330.0
cX,cY: 13 20
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/5789.tif
Blur image with bilat and median filter
xmean,ymean: 22 45
area: 574.0
cX,cY: 22 45
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7716.tif
Blur image with bilat and median filter
xmean,ymean: 28 19
area: 2.0
area: 8.5
area: 2.0
area: 5.5
area: 95

Blur image with bilat and median filter
xmean,ymean: 18 23
area: 119.0
area: 3.5
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7115.tif
Blur image with bilat and median filter
xmean,ymean: 64 35
area: 113.5
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/6716.tif
Blur image with bilat and median filter
xmean,ymean: 23 35
area: 0.0
area: 3.5
area: 16.0
area: 5.0
area: 15.5
area: 0.0
area: 0.0
area: 38.5
area: 460.5
cX,cY: 33 52
area: 0.0
area: 24.5
area: 0.5
area: 0.0
area: 0.0
area: 0.0
area: 0.0
area: 925.0
cX,cY: 15 23
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/3742.tif
Blur image with bilat and median filter
xmean,ymean: 44 15
area: 26.5
area: 0.0
area: 15.0
area: 276.0
cX,cY: 52 9
area: 24.5
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasse

//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/8611.tif
Blur image with bilat and median filter
xmean,ymean: 52 42
area: 267.0
cX,cY: 52 42
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/2763.tif
Blur image with bilat and median filter
xmean,ymean: 37 37
area: 0.0
area: 95.5
area: 0.0
area: 0.0
area: 0.0
area: 0.5
area: 0.0
area: 2541.0
cX,cY: 37 36
area: 7.5
area: 7.5
area: 0.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7934.tif
Blur image with bilat and median filter
xmean,ymean: 19 45
area: 658.5
cX,cY: 18 47
area: 0.0
area: 0.0
area: 11.5
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/3828.tif
Blur image with bilat and median filter
xmean,ymean: 40 26
area: 24.5
area: 228.0
cX,cY: 56 41
area: 137.0
cX,cY: 7

xmean,ymean: 41 54
area: 93.0
area: 0.0
area: 10.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/2273.tif
Blur image with bilat and median filter
xmean,ymean: 13 42
area: 5.5
area: 239.5
cX,cY: 12 41
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/3385.tif
Blur image with bilat and median filter
xmean,ymean: 5 4
area: 108.5
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/7929.tif
Blur image with bilat and median filter
xmean,ymean: 33 54
area: 0.0
area: 43.5
area: 3.0
area: 431.5
cX,cY: 47 56
area: 202.0
cX,cY: 9 50
area: 0.0
//ibs9010/current_data/Data_Keshav/image/Classification_model/Inference/BinaryClasses/singleneurontiles/MG48_bs/S51/6825.tif
Blur image with bilat and median filter
xmean,ymean: 58 55
area: 619.0
cX,cY: 58 55
//ibs9010/current_data/Data_Keshav/image/Classificati